In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
from tqdm.notebook import tqdm

In [394]:
#Проверка связи с источником
import requests  

url = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&offset=0'
response = requests.get(url)
response

<Response [200]>

Поскольку код сайта достаточно массивный и запутанный, некоторые данные не представляется возможным вытаскивать напрямую. В связи с этим, создадим функцию, которая будет преобразовывать получаемые строки в нужный нам вид.

In [136]:
def convertor(strings):
    strings = str(strings)
    st = strings.split('>')
    strings = st[1].split('<')[0]
    return strings
#text = '<p class="_5oAgZI Z4r7pA">9 012 283 ₽</p' # Можно запустить код и посмотреть, как он работает на примере
#convertor(text)

'9 012 283 ₽'

Так, следующая ячейка создаст разочарование на лице проверяющего и сильное желание снизить балл за красоту или ещё что-нибудь) 

Собственно, эти ссылки затрагивают все страницы, из которых в дальнейшем будут парситься данные. 

У ассиста возникает логичный вопрос: что ж ты, тупая башка, просто не закинул их в функцию, где спокойно бы менял запросы???
    
    
Объясняю) Дело в том, что маркетологи ДомКлика - ребята с приколом. Когда ты выставляешь фильтр на минимальную цену квартиры, условно, в 10 миллионов рублей, тебе сначала выдадут предложений 300, где цена будет "ОТ 7-9 миллионов". В связи с этим возникает перехлёст данных и вероятность дубликатов. 

Так что для каждой ссылки мы ручками подбирали номер страницы, начиная с которой предложения идут реально от выбранной цены.

Поэтому просим понять и пхростить!

In [408]:
url_0 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&sort=price&sort_dir=asc&offset={}'
url_1 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=6950000&offset={}'
url_2 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=8150000&offset={}'
url_3 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=8950000&offset={}'
url_4 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=9650000&offset={}'
url_5 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=10250000&offset={}'
url_6 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=10850000&offset={}'
url_7 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=11450000&offset={}'
url_8 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=12000000&offset={}'
url_9 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=12550000&offset={}'
url_10 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=13190000&offset={}'
url_11 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=13840000&offset={}'
url_12 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=14510000&offset={}'
url_13 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=15260000&offset={}'
url_14 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=16010000&offset={}'
url_15 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=16920000&offset={}'
url_16 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=17900000&offset={}'
url_17 = 'https://domclick.ru/search?deal_type=sale&category=living&offer_type=flat&offer_type=layout&aids=2299&sale_price__lte=20000000&utm_referrer=https%3A%2F%2Fdomclick.ru%2Fauth%2FloginPage&sort=price&sort_dir=asc&sale_price__gte=18925000&offset={}'


Теперь можем создать функцию от 2-х переменных, с помощью которой будет парсить наши данные:

In [410]:
from bs4 import BeautifulSoup

def get_page(url, p):
    
    url = url.format(p) # переходим по ссылке
    
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser') # строим дерево
    
    # находи в нём все необходимые параметры
    flats = tree.find_all('span', {'class' : '_6KKuHL iaB6kz'})
    flats_price = tree.find_all('p', {'class' : '_5oAgZI Z4r7pA'})
    flats_adress = tree.find_all('span', {'class' : 'S4xsAI'})
    info = []
    
    for f in range(len(flats)):
        if f%3 == 0:
            f_st_0 = convertor(flats[f])
        elif f%3 == 1:
            f_st_1 = convertor(flats[f])
        elif f%3 == 2:
            info.append({'Тип': f_st_0,
                     'Площадь': f_st_1,
                     'Этаж': convertor(flats[f]),
                    'Стоимость': convertor(flats_price[f//3]),
                         'Адрес': convertor(flats_adress[f//3])})
    return info

In [409]:
get_page(url_17, 380)

[<p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>, <p class="_5oAgZI Z4r7pA">19 000 000 ₽</p>]


[{'Тип': '2-комн. апартаменты',
  'Площадь': '54,29 м²',
  'Этаж': '8/21 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, Автомобильный проезд, 4'},
 {'Тип': '1-комн. квартира',
  'Площадь': '37,7 м²',
  'Этаж': '24/29 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, Аминьевское шоссе, 4Дк2'},
 {'Тип': '3-комн. квартира',
  'Площадь': '76 м²',
  'Этаж': '3/8 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, Бутырская улица, 53к1'},
 {'Тип': '1-комн. квартира',
  'Площадь': '48,1 м²',
  'Этаж': '2/24 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, улица Маршала Тухачевского, 49'},
 {'Тип': '3-комн. квартира',
  'Площадь': '96 м²',
  'Этаж': '3/19 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, Московский, 3-й микрорайон, 13'},
 {'Тип': '3-комн. квартира',
  'Площадь': '77 м²',
  'Этаж': '12/17 эт.',
  'Стоимость': '19 000 000 ₽',
  'Адрес': 'Москва, улица Грина, 24'},
 {'Тип': '2-комн. квартира',
  'Площадь': '56 м²',
  'Этаж': '5/6 эт.',
  'Стои

Теперь проходимся циклом по страницам и закидываем найденные данные в списки, которые далее конвертируем в табличку.

Тут второй момент, когда гремаса ассиста переходит в скупую слезу из-за большого количества строк) Конечно, можно было бы всё сделать в одной ячейке, но, во-первых, это тоже не сильно приятно глазу и по стандарту PEP-8, длина строки не может превышать 72 символов, а во-вторых, так легче искать косяки. Не бейте

In [411]:
info_0 = [] 

for p in tqdm(range(0,1981,20)):
    info_0.extend(get_page(url_0,p))


  0%|          | 0/100 [00:00<?, ?it/s]

In [474]:
import pandas as pd

df_0 = pd.DataFrame(info_0)
print(df_0.shape)
df_0.head()

(2000, 5)


,Тип,Площадь,Этаж,Стоимость,Адрес
0,Студия,10 м²,1/3 эт.,1 049 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
1,Студия,10 м²,1/3 эт.,1 150 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
2,Апартаменты-студия,"11,7 м²",1/16 эт.,1 500 000 ₽,"Москва, Юго-Восточный административный округ, ..."
3,Апартаменты-студия,"15,3 м²",1/16 эт.,1 590 000 ₽,"Москва, Волжский бульвар, 19"
4,Апартаменты-студия,13 м²,1/3 эт.,1 650 000 ₽,"Москва, посёлок Первомайское, Центральная улиц..."


In [413]:
info_1 = [] 

for p in tqdm(range(120,1981,20)):
    info_1.extend(get_page(url_1,p))

df_1 = pd.DataFrame(info_1)

  0%|          | 0/94 [00:00<?, ?it/s]

In [414]:
info_2 = [] 

for p in tqdm(range(220,1981,20)):
    info_2.extend(get_page(url_2,p))
    
df_2 = pd.DataFrame(info_2)

  0%|          | 0/89 [00:00<?, ?it/s]

In [415]:
info_3 = [] 

for p in tqdm(range(280,1981,20)):
    info_3.extend(get_page(url_3,p))

df_3 = pd.DataFrame(info_3)

  0%|          | 0/86 [00:00<?, ?it/s]

In [416]:
info_4 = [] 

for p in tqdm(range(300,1981,20)):
    info_4.extend(get_page(url_4,p))
    
df_4 = pd.DataFrame(info_4)

  0%|          | 0/85 [00:00<?, ?it/s]

In [417]:
info_5 = [] 

for p in tqdm(range(340,1981,20)):
    info_5.extend(get_page(url_5,p))
df_5 = pd.DataFrame(info_5)

  0%|          | 0/83 [00:00<?, ?it/s]

In [418]:
info_6 = [] 

for p in tqdm(range(380,1981,20)):
    info_6.extend(get_page(url_6,p))
    
df_6 = pd.DataFrame(info_6)

  0%|          | 0/81 [00:00<?, ?it/s]

In [419]:
info_7 = [] 

for p in tqdm(range(340,1981,20)):
    info_7.extend(get_page(url_7,p))
df_7 = pd.DataFrame(info_7)

  0%|          | 0/83 [00:00<?, ?it/s]

In [420]:
info_8 = [] 

for p in tqdm(range(360,1981,20)):
    info_8.extend(get_page(url_8,p))
df_8 = pd.DataFrame(info_8)

  0%|          | 0/82 [00:00<?, ?it/s]

In [421]:
info_9 = [] 

for p in tqdm(range(360,1981,20)):
    info_9.extend(get_page(url_9,p))
df_9 = pd.DataFrame(info_9)

  0%|          | 0/82 [00:00<?, ?it/s]

In [422]:
info_10 = [] 

for p in tqdm(range(360,1981,20)):
    info_10.extend(get_page(url_10,p))
df_10 = pd.DataFrame(info_10)

  0%|          | 0/82 [00:00<?, ?it/s]

In [423]:
info_11 = [] 

for p in tqdm(range(320,1981,20)):
    info_11.extend(get_page(url_11,p))
df_11 = pd.DataFrame(info_11)

  0%|          | 0/84 [00:00<?, ?it/s]

In [424]:
info_12 = [] 

for p in tqdm(range(340,1981,20)):
    info_12.extend(get_page(url_12,p))
df_12 = pd.DataFrame(info_12)

  0%|          | 0/83 [00:00<?, ?it/s]

In [425]:
info_13 = [] 

for p in tqdm(range(320,1981,20)):
    info_13.extend(get_page(url_13,p))
df_13 = pd.DataFrame(info_13)

  0%|          | 0/84 [00:00<?, ?it/s]

In [426]:
info_14 = [] 

for p in tqdm(range(300,1981,20)):
    info_14.extend(get_page(url_14,p))
df_14 = pd.DataFrame(info_14)

  0%|          | 0/85 [00:00<?, ?it/s]

In [427]:
info_15 = [] 

for p in tqdm(range(280,1981,20)):
    info_15.extend(get_page(url_15,p))
df_15 = pd.DataFrame(info_15)

  0%|          | 0/86 [00:00<?, ?it/s]

In [428]:
info_16 = [] 

for p in tqdm(range(260,1981,20)):
    info_16.extend(get_page(url_16,p))
df_16 = pd.DataFrame(info_16)

  0%|          | 0/87 [00:00<?, ?it/s]

In [429]:
info_17 = [] 

for p in tqdm(range(240,1981,20)):
    info_17.extend(get_page(url_17,p))
df_17 = pd.DataFrame(info_17)

  0%|          | 0/88 [00:00<?, ?it/s]

In [430]:
df_17.tail()

,Тип,Площадь,Этаж,Стоимость,Адрес
1755,3-комн. квартира,"59,7 м²",1/5 эт.,20 000 000 ₽,"Москва, Ботаническая улица, 5"
1756,2-комн. квартира,"60,8 м²",4/22 эт.,20 000 000 ₽,"Москва, 3-я Хорошёвская улица, 17А"
1757,2-комн. квартира,52 м²,13/17 эт.,20 000 000 ₽,"Москва, улица Борисовские Пруды, 13к1"
1758,3-комн. квартира,"78,5 м²",7/13 эт.,20 000 000 ₽,"Москва, Южнобутовская улица, 71"
1759,1-комн. квартира,"41,5 м²",4/9 эт.,20 000 000 ₽,"Москва, Автозаводская улица, 23с64"


Ура, мучения закончились, встаньте из-за стола, сделайте разминку для глаз, и продолжим)

Далее мы объединяем все таблички в одну и сохраняем её в отдельный файл, ибо ответ "200" выдаётся вообще не всегда:/

In [431]:
df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12, df_13, df_14, df_15, df_16, df_17])
df.reset_index(drop=True, inplace=True)
df.to_csv('Project_data.txt', sep=',', index=False)
df


,Тип,Площадь,Этаж,Стоимость,Адрес
0,Студия,10 м²,1/3 эт.,1 049 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
1,Студия,10 м²,1/3 эт.,1 150 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
2,Апартаменты-студия,"11,7 м²",1/16 эт.,1 500 000 ₽,"Москва, Юго-Восточный административный округ, ..."
3,Апартаменты-студия,"15,3 м²",1/16 эт.,1 590 000 ₽,"Москва, Волжский бульвар, 19"
4,Апартаменты-студия,13 м²,1/3 эт.,1 650 000 ₽,"Москва, посёлок Первомайское, Центральная улиц..."
...,...,...,...,...,...
30795,3-комн. квартира,"59,7 м²",1/5 эт.,20 000 000 ₽,"Москва, Ботаническая улица, 5"
30796,2-комн. квартира,"60,8 м²",4/22 эт.,20 000 000 ₽,"Москва, 3-я Хорошёвская улица, 17А"
30797,2-комн. квартира,52 м²,13/17 эт.,20 000 000 ₽,"Москва, улица Борисовские Пруды, 13к1"
30798,3-комн. квартира,"78,5 м²",7/13 эт.,20 000 000 ₽,"Москва, Южнобутовская улица, 71"


Всё, теперь мы окончательно не зависим от сайта, можно смело влепить им диз и юзать Циан. 

В последующих шагах мы причёсываем данные, приводим значения в правильный читаемый формат. Может возникнуть вопрос лишь по одному решению:
    
Все цены, которые не имеют точного значения, то есть записаны в формате: "От X рублей" мы удаляем и придаём им ближайшие предыдущие значения. Почему так? 
    
Ну, данных форматов достаточно много и просто не принимать их во внимание нельзя. Поскольку цена не задана точно, возникает вопрос, какое значение ей придавать. Наше решение брать минимальную возможную сумму достаточно логично и просто, и скорее всего применимо в реальной жизни. Вряд ли, откликаясь на объявление "Продаётся студия от 5 млн." человек рассматривает вариант, что отдаст 10 или хотя бы 6. Второй вопрос, а нормально ли придавать значения предыдущих значений? Да, вполне, потому что в силу плотности данных, максимальная разница может быть, ну, от силы, 50 тысяч, хотя в большинстве случаев её вообще нет. Эта сумма абсолютно некритична и не исказит результаты, так как почти люой продавец уступит 0,5% стоимости.

In [432]:
df = df.replace('от ', np.nan)
df

,Тип,Площадь,Этаж,Стоимость,Адрес
0,Студия,10 м²,1/3 эт.,1 049 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
1,Студия,10 м²,1/3 эт.,1 150 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
2,Апартаменты-студия,"11,7 м²",1/16 эт.,1 500 000 ₽,"Москва, Юго-Восточный административный округ, ..."
3,Апартаменты-студия,"15,3 м²",1/16 эт.,1 590 000 ₽,"Москва, Волжский бульвар, 19"
4,Апартаменты-студия,13 м²,1/3 эт.,1 650 000 ₽,"Москва, посёлок Первомайское, Центральная улиц..."
...,...,...,...,...,...
30795,3-комн. квартира,"59,7 м²",1/5 эт.,20 000 000 ₽,"Москва, Ботаническая улица, 5"
30796,2-комн. квартира,"60,8 м²",4/22 эт.,20 000 000 ₽,"Москва, 3-я Хорошёвская улица, 17А"
30797,2-комн. квартира,52 м²,13/17 эт.,20 000 000 ₽,"Москва, улица Борисовские Пруды, 13к1"
30798,3-комн. квартира,"78,5 м²",7/13 эт.,20 000 000 ₽,"Москва, Южнобутовская улица, 71"


In [433]:
df['Стоимость'] = df['Стоимость'].fillna(method='ffill')

df.rename(columns={'Площадь': 'Площадь, м²', 'Стоимость': 'Стоимость, ₽', 'Этаж': 'Номер этажа'}, inplace=True)

df

,Тип,"Площадь, м²",Номер этажа,"Стоимость, ₽",Адрес
0,Студия,10 м²,1/3 эт.,1 049 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
1,Студия,10 м²,1/3 эт.,1 150 000 ₽,"Москва, поселение Первомайское, деревня Иванов..."
2,Апартаменты-студия,"11,7 м²",1/16 эт.,1 500 000 ₽,"Москва, Юго-Восточный административный округ, ..."
3,Апартаменты-студия,"15,3 м²",1/16 эт.,1 590 000 ₽,"Москва, Волжский бульвар, 19"
4,Апартаменты-студия,13 м²,1/3 эт.,1 650 000 ₽,"Москва, посёлок Первомайское, Центральная улиц..."
...,...,...,...,...,...
30795,3-комн. квартира,"59,7 м²",1/5 эт.,20 000 000 ₽,"Москва, Ботаническая улица, 5"
30796,2-комн. квартира,"60,8 м²",4/22 эт.,20 000 000 ₽,"Москва, 3-я Хорошёвская улица, 17А"
30797,2-комн. квартира,52 м²,13/17 эт.,20 000 000 ₽,"Москва, улица Борисовские Пруды, 13к1"
30798,3-комн. квартира,"78,5 м²",7/13 эт.,20 000 000 ₽,"Москва, Южнобутовская улица, 71"


In [434]:
df['Количество этажей'] = df['Номер этажа'] 

for i in range(df.shape[0]):
    
    df['Стоимость, ₽'][i] = int(df['Стоимость, ₽'][i][:-2].replace(' ', ''))
    df['Площадь, м²'][i] = float(df['Площадь, м²'][i][:-3].replace(',', '.'))
    df['Количество этажей'][i] = int(df['Количество этажей'][i][:-4].split('/')[1])
    
    if '-' not in df['Номер этажа'][i]:
        df['Номер этажа'][i] = int(df['Номер этажа'][i][:-4].split('/')[0])
    else:
        df['Номер этажа'][i] = int(df['Номер этажа'][i][:-4].split('/')[0].split('-')[1])
    
df

,Тип,"Площадь, м²",Номер этажа,"Стоимость, ₽",Адрес,Количество этажей
0,Студия,10.0,1,1049000,"Москва, поселение Первомайское, деревня Иванов...",3
1,Студия,10.0,1,1150000,"Москва, поселение Первомайское, деревня Иванов...",3
2,Апартаменты-студия,11.7,1,1500000,"Москва, Юго-Восточный административный округ, ...",16
3,Апартаменты-студия,15.3,1,1590000,"Москва, Волжский бульвар, 19",16
4,Апартаменты-студия,13.0,1,1650000,"Москва, посёлок Первомайское, Центральная улиц...",3
...,...,...,...,...,...,...
30795,3-комн. квартира,59.7,1,20000000,"Москва, Ботаническая улица, 5",5
30796,2-комн. квартира,60.8,4,20000000,"Москва, 3-я Хорошёвская улица, 17А",22
30797,2-комн. квартира,52.0,13,20000000,"Москва, улица Борисовские Пруды, 13к1",17
30798,3-комн. квартира,78.5,7,20000000,"Москва, Южнобутовская улица, 71",13


In [435]:
df['Стоимость, ₽'] = df['Стоимость, ₽'].astype(int)
df['Площадь, м²'] = df['Площадь, м²'].astype(float)

df['Стоимость м², ₽/м²'] = df['Стоимость, ₽'] / df['Площадь, м²']
df['Стоимость м², ₽/м²'] = df['Стоимость м², ₽/м²'].round()

df

,Тип,"Площадь, м²",Номер этажа,"Стоимость, ₽",Адрес,Количество этажей,"Стоимость м², ₽/м²"
0,Студия,10.0,1,1049000,"Москва, поселение Первомайское, деревня Иванов...",3,104900.0
1,Студия,10.0,1,1150000,"Москва, поселение Первомайское, деревня Иванов...",3,115000.0
2,Апартаменты-студия,11.7,1,1500000,"Москва, Юго-Восточный административный округ, ...",16,128205.0
3,Апартаменты-студия,15.3,1,1590000,"Москва, Волжский бульвар, 19",16,103922.0
4,Апартаменты-студия,13.0,1,1650000,"Москва, посёлок Первомайское, Центральная улиц...",3,126923.0
...,...,...,...,...,...,...,...
30795,3-комн. квартира,59.7,1,20000000,"Москва, Ботаническая улица, 5",5,335008.0
30796,2-комн. квартира,60.8,4,20000000,"Москва, 3-я Хорошёвская улица, 17А",22,328947.0
30797,2-комн. квартира,52.0,13,20000000,"Москва, улица Борисовские Пруды, 13к1",17,384615.0
30798,3-комн. квартира,78.5,7,20000000,"Москва, Южнобутовская улица, 71",13,254777.0


In [475]:
if df.isnull().sum().sum() == 0:
    print('Пропусков нет')
else:
    print('Пропуски есть')


Пропусков нет


In [3]:
# Список для хранения координат
coordinates_list = []

for i in tqdm(range(100)):
    if df_copy['Адрес'][i] not in coordinates_list:
        k = df_copy['Адрес'][i]
        # Формируем запрос к API Photon
        response = requests.get(f'https://photon.komoot.io/api/?q={k}')
    
        # Проверяем статус ответа
        if response.status_code == 200:
            data = response.json()
            # Извлекаем координаты первого результата
            features = data.get('features')
            if features:
                coordinates = features[0].get('geometry').get('coordinates')
                coordinates_list.append({k: coordinates})
            else:
                coordinates_list.append((None, None))
        else:
            coordinates_list.append((None, None))
    else:
        pass

print(coordinates_list)


  0%|          | 0/100 [00:00<?, ?it/s]

NameError: name 'df_copy' is not defined

In [469]:
for j in tqdm(range(df_copy.shape[0])):
    df_copy['Адрес'][j]= ', '.join(df_copy['Адрес'][j].split(', ')[:-1])
df_copy

  0%|          | 0/30800 [00:00<?, ?it/s]

/var/folders/2c/02h9jm4503b60m2cbcyj7_x00000gn/T/ipykernel_54270/3886777411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['Адрес'][j]= ', '.join(df_copy['Адрес'][j].split(', ')[:-1])


,Тип,"Площадь, м²",Номер этажа,"Стоимость, ₽",Адрес,Количество этажей,"Стоимость м², ₽/м²"
0,Студия,10.0,1,1049000,"Москва, поселение Первомайское, деревня Иванов...",3,104900.0
1,Студия,10.0,1,1150000,"Москва, поселение Первомайское, деревня Иванов...",3,115000.0
2,Апартаменты-студия,11.7,1,1500000,"Москва, Юго-Восточный административный округ, ...",16,128205.0
3,Апартаменты-студия,15.3,1,1590000,"Москва, Волжский бульвар",16,103922.0
4,Апартаменты-студия,13.0,1,1650000,"Москва, посёлок Первомайское, Центральная улица",3,126923.0
...,...,...,...,...,...,...,...
30795,3-комн. квартира,59.7,1,20000000,"Москва, Ботаническая улица",5,335008.0
30796,2-комн. квартира,60.8,4,20000000,"Москва, 3-я Хорошёвская улица",22,328947.0
30797,2-комн. квартира,52.0,13,20000000,"Москва, улица Борисовские Пруды",17,384615.0
30798,3-комн. квартира,78.5,7,20000000,"Москва, Южнобутовская улица",13,254777.0
